In [96]:
import requests
import json
import geocoder
import numpy as np
import pandas as pd

In [6]:
g = geocoder.bing('Brooklyn, NY', key='AotgMikv-IbXolSLqnnduQm27CVjN0zihKAgLhEmic-z913hSfwZB0UjeDXYJulD')

In [84]:
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
  client_id = 'JJZPRPAN3YE1FTOG3NXK253ROHHNCQ5IU2GHXGUV4IS0VRCI', # your Foursquare ID
  client_secret = 'O3WFKOGCMPG333GLSTRNEQ2LV2UMGL1AJZGZBQWGDMLLFZMK', # your Foursquare Secret
  v = '20180605', # Foursquare API version
  ll = str(g.json['lat']) + ',' + str(g.json['lng']),
  query = 'restaurant',
  radius = 1000,
  limit = 50,
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [87]:
restaurants = []
for m in data['response']['groups'][0]['items']:
    restaurants.append(m['venue']['name'])

In [106]:
d = {}
d['users'] = ['user'+str(i) for i in range(20)]
d['good restaurants'] = []
d['bad restaurants'] = []
for i in range(1,21):
    restaurant_list = np.random.choice(restaurants, size=30, replace=False)
    d['good restaurants'].append(restaurant_list[:15])
    d['bad restaurants'].append(restaurant_list[15:])

In [107]:
user_df = pd.DataFrame.from_dict(d)

In [108]:
user_df = user_df[['users','good restaurants','bad restaurants']]

In [109]:
user_df.head()

,users,good restaurants,bad restaurants
0,user0,"[Fast and Fresh Burrito Deli, DeKalb Market Ha...","[Paris Baguette, Mile End Delicatessen, Court ..."
1,user1,"[Court Street Bagels, Chez Moi, One Girl Cooki...","[Wild Ginger, Beasts & Bottles, Colonie, Henry..."
2,user2,"[Bar Tabac, Bien Cuit, Dellarocco's, Beasts & ...","[The Gumbo Bros, Yemen Cafe, My Little Pizzeri..."
3,user3,"[Pio Bagel, Grand Army, One Girl Cookies, Mile...","[DeKalb Market Hall, Court Street Bagels, Five..."
4,user4,"[two8two Bar & Burger, Chipotle Mexican Grill,...","[Doner Kebab NYC, Maison Kayser, French Louie,..."


In [114]:
df_lunch_group = user_df.sample(n=6)

In [122]:
df_lunch_group = df_lunch_group.sort_index()

In [123]:
df_lunch_group

,users,good restaurants,bad restaurants
0,user0,"[Fast and Fresh Burrito Deli, DeKalb Market Ha...","[Paris Baguette, Mile End Delicatessen, Court ..."
3,user3,"[Pio Bagel, Grand Army, One Girl Cookies, Mile...","[DeKalb Market Hall, Court Street Bagels, Five..."
10,user10,"[Five Guys, Wild Ginger, Luzzo's BK, Saketumi ...","[Court Street Bagels, Bar Tabac, Rocco's Tacos..."
11,user11,"[Colonie, Table 87, Dos Toros Taquería, Gregor...","[Paris Baguette, Grand Army, Saketumi Asian Bi..."
15,user15,"[Shelsky's of Brooklyn, Rocco's Tacos and Tequ...","[DeKalb Market Hall, Five Guys, Chipotle Mexic..."
18,user18,"[My Little Pizzeria, Hibino, Shelsky's of Broo...","[Wild Ginger, Court Street Bagels, The Gumbo B..."


In [133]:
lunch_votes = [(restaurant,0) for restaurant in restaurants]

In [134]:
lunch_dict = dict(lunch_votes)

In [135]:
lunch_dict

{'Bar Tabac': 0,
 'Bareburger': 0,
 'Beasts & Bottles': 0,
 'Bien Cuit': 0,
 'Chez Moi': 0,
 'Chipotle Mexican Grill': 0,
 'Colonie': 0,
 'Court Street Bagels': 0,
 'Damascus Bread & Pastry Shop': 0,
 'DeKalb Market Hall': 0,
 "Dellarocco's": 0,
 'Doner Kebab NYC': 0,
 'Dos Toros Taquería': 0,
 'Fast and Fresh Burrito Deli': 0,
 'Five Guys': 0,
 'French Louie': 0,
 'Grand Army': 0,
 "Gregory's Coffee": 0,
 'Henry Public': 0,
 'Hibino': 0,
 'Iron Chef House': 0,
 'Ki Sushi': 0,
 'La Bagel Delight': 0,
 'La Vara': 0,
 'Lassen & Hennigs': 0,
 "Luzzo's BK": 0,
 'Maison Kayser': 0,
 'Mile End Delicatessen': 0,
 'My Little Pizzeria': 0,
 'One Girl Cookies': 0,
 'Panera Bread': 0,
 'Paris Baguette': 0,
 'Pio Bagel': 0,
 'Piz-zetta': 0,
 'Queen': 0,
 'River Deli': 0,
 "Rocco's Tacos and Tequila Bar Brooklyn": 0,
 'Saketumi Asian Bistro': 0,
 "Shelsky's of Brooklyn": 0,
 'Sophies Cuban Cuisine': 0,
 'Sottocasa Pizzeria': 0,
 'Sushi Gallery': 0,
 'Table 87': 0,
 'The Atlantic ChipShop': 0,
 'The

In [136]:
for rest in lunch_dict.keys():
    for good_choice in df_lunch_group['good restaurants']:
        if rest in good_choice:
            lunch_dict[rest] += 1
    for bad_choice in df_lunch_group['bad restaurants']:
        if rest in bad_choice:
            lunch_dict[rest] -= 1